# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [59]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [60]:
weather_df = pd.read_csv("cities.csv")
#added this live to remove Unnamed 0 column
del weather_df["Unnamed: 0"]
weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Stykkisholmur,65.0757,-22.7297,56.05,78,52,4.32,IS,1625943795
1,Vaini,-21.2000,-175.2000,70.86,77,100,11.56,TO,1625943797
2,Ushuaia,-54.8000,-68.3000,35.26,93,75,16.11,AR,1625943649
3,Punta Arenas,-53.1500,-70.9167,33.91,86,20,19.57,CL,1625943799
4,Saint George,37.1041,-113.5841,114.64,12,1,1.01,US,1625943800
...,...,...,...,...,...,...,...,...,...
593,Pontes e Lacerda,-15.2261,-59.3353,93.99,16,17,4.32,BR,1625944603
594,Imbituba,-28.2400,-48.6703,74.82,74,0,5.17,BR,1625944605
595,Shitanjing,39.2342,106.3439,73.60,51,100,5.68,CN,1625944606
596,Arlit,18.7369,7.3853,96.12,22,77,6.40,NE,1625944607


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [73]:
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)
fig = gmaps.figure()
 
#adding heat map with max intensiy
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [65]:
#added humidity to dataset to limit data.
perfect_weather = weather_df.loc[(weather_df["Max Temp"]>=70)&(weather_df["Max Temp"]<=80)&
                                (weather_df["Cloudiness"]==0)&(weather_df["Wind Speed"]<10)&
                                 (weather_df["Humidity"]<=60)].dropna()
perfect_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,Kalemie,-5.9475,29.1947,70.16,50,0,4.92,CD,1625943813
147,Chilliwack,49.1747,-121.9443,78.58,55,0,8.01,CA,1625943995
172,Shelburne,44.0787,-80.2041,75.76,52,0,3.31,CA,1625944028
320,Carmo do Paranaíba,-19.0008,-46.3161,74.75,33,0,1.88,BR,1625944241
322,Kikwit,-5.0410,18.8162,78.24,26,0,2.10,CD,1625944243
356,Ürümqi,43.8010,87.6005,74.35,57,0,6.71,CN,1625944290
426,Gazojak,41.1875,61.4036,77.77,38,0,6.15,TM,1625944383
438,Verneşti,45.2167,26.7333,72.27,51,0,3.74,RO,1625944399
445,Vila,42.0304,-8.1588,77.32,55,0,3.89,PT,1625944408
451,Shubarkuduk,49.1447,56.4872,77.13,38,0,7.31,KZ,1625944415


In [ ]:
#5 to 10 cities max

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [66]:
hotel_df = perfect_weather
hotel_df ["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
14,Kalemie,-5.9475,29.1947,70.16,50,0,4.92,CD,1625943813,
147,Chilliwack,49.1747,-121.9443,78.58,55,0,8.01,CA,1625943995,
172,Shelburne,44.0787,-80.2041,75.76,52,0,3.31,CA,1625944028,
320,Carmo do Paranaíba,-19.0008,-46.3161,74.75,33,0,1.88,BR,1625944241,
322,Kikwit,-5.0410,18.8162,78.24,26,0,2.10,CD,1625944243,
356,Ürümqi,43.8010,87.6005,74.35,57,0,6.71,CN,1625944290,
426,Gazojak,41.1875,61.4036,77.77,38,0,6.15,TM,1625944383,
438,Verneşti,45.2167,26.7333,72.27,51,0,3.74,RO,1625944399,
445,Vila,42.0304,-8.1588,77.32,55,0,3.89,PT,1625944408,
451,Shubarkuduk,49.1447,56.4872,77.13,38,0,7.31,KZ,1625944415,


In [67]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat and lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params = {
    "location": f"{lat}, {lng}",  
    "radius": 5000,
    "type": "Hotel",
    "key": g_key,}

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
   
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        
           
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    
hotel_df

Retrieving Results for Index 14: Kalemie.
Closest hotel in Kalemie is Kalemie.
Retrieving Results for Index 147: Chilliwack.
Closest hotel in Chilliwack is Chilliwack.
Retrieving Results for Index 172: Shelburne.
Closest hotel in Shelburne is Mono.
Retrieving Results for Index 320: Carmo do Paranaíba.
Closest hotel in Carmo do Paranaíba is Carmo do Paranaíba.
Retrieving Results for Index 322: Kikwit.
Closest hotel in Kikwit is Kikwit.
Retrieving Results for Index 356: Ürümqi.
Closest hotel in Ürümqi is Ürümqi.
Retrieving Results for Index 426: Gazojak.
Closest hotel in Gazojak is Pitnak.
Retrieving Results for Index 438: Verneşti.
Closest hotel in Verneşti is Buzău.
Retrieving Results for Index 445: Vila.
Closest hotel in Vila is Castro Laboreiro.
Retrieving Results for Index 451: Shubarkuduk.
Closest hotel in Shubarkuduk is Shubarkuduk.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
14,Kalemie,-5.9475,29.1947,70.16,50,0,4.92,CD,1625943813,Kalemie
147,Chilliwack,49.1747,-121.9443,78.58,55,0,8.01,CA,1625943995,Chilliwack
172,Shelburne,44.0787,-80.2041,75.76,52,0,3.31,CA,1625944028,Mono
320,Carmo do Paranaíba,-19.0008,-46.3161,74.75,33,0,1.88,BR,1625944241,Carmo do Paranaíba
322,Kikwit,-5.0410,18.8162,78.24,26,0,2.10,CD,1625944243,Kikwit
356,Ürümqi,43.8010,87.6005,74.35,57,0,6.71,CN,1625944290,Ürümqi
426,Gazojak,41.1875,61.4036,77.77,38,0,6.15,TM,1625944383,Pitnak
438,Verneşti,45.2167,26.7333,72.27,51,0,3.74,RO,1625944399,Buzău
445,Vila,42.0304,-8.1588,77.32,55,0,3.89,PT,1625944408,Castro Laboreiro
451,Shubarkuduk,49.1447,56.4872,77.13,38,0,7.31,KZ,1625944415,Shubarkuduk


In [68]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [69]:
markers = gmaps.marker_layer(locations,
    info_box_content= hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))